In [12]:
class Monad:
    def __init__(self, value):
        self.value = value
        self.status = 'dirty'  # Starting with 'dirty'
        self.message = []
        self.dtype = object
    def __or__(self, func):
        if self.status == 'dirty': # Only process if 'dirty'
            try:
                x = func(self.value)
                self.value = x
                self.status = 'passed'
                self.message.append(f'Passed:{func.__name__}()')
                return self 
            except Exception as e:
                self.message.append(f'Failed:{func.__name__}():{str(e)}')
        return self

    def __repr__(self):
        return f'{self.status}({self.value}){self.message}'

    def __str__(self):
        return self.__repr__()

In [13]:
def country_code(x):
    valid_countries = ['TH', 'US', 'UK']
    if x not in valid_countries:
        raise Exception("Invalid code")
    else:
        return x
        
def country_name(x):
    valid_countries ={'Thailand':'TH','United State':'USA', 'United Kingdom':'UK'}
    return valid_countries[x]

def country_name_thai(x):
    valid_countries ={'ไทย':'TH','สหรัฐอเมริกา':'USA', 'อังกฤษ':'UK'}
    return valid_countries[x]

In [14]:
m=Monad('ไทย.')|country_code|country_name|country_name_thai
m

dirty(ไทย.)['Failed:country_code():Invalid code', "Failed:country_name():'ไทย.'", "Failed:country_name_thai():'ไทย.'"]

In [15]:
import pandas as pd
df=pd.DataFrame({'country':['TH','United State','อังกฤษ','invalid name']})
df['result']=df['country'].apply(lambda x: Monad(x)|country_code|country_name|country_name_thai )
# df=df.reset_index(names=['row'])
df

,country,result
0,TH,passed(TH)['Passed:country_code()']
1,United State,passed(USA)['Failed:country_code():Invalid cod...
2,อังกฤษ,passed(UK)['Failed:country_code():Invalid code...
3,invalid name,dirty(invalid name)['Failed:country_code():Inv...


In [18]:
df_result=pd.DataFrame({
    'input':df['country'],
    'value':df['result'].apply(lambda x: x.value),
    # 'status':df['result'].apply(lambda x: x.status),
    'message':df['result'].apply(lambda x: x.message),
    
})
df_result['column']='country'
df_result

,input,value,message,column
0,TH,TH,[Passed:country_code()],country
1,United State,USA,"[Failed:country_code():Invalid code, Passed:co...",country
2,อังกฤษ,UK,"[Failed:country_code():Invalid code, Failed:co...",country
3,invalid name,invalid name,"[Failed:country_code():Invalid code, Failed:co...",country


In [19]:
df_result.explode('message').reset_index(names=['row'])

,row,input,value,message,column
0,0,TH,TH,Passed:country_code(),country
1,1,United State,USA,Failed:country_code():Invalid code,country
2,1,United State,USA,Passed:country_name(),country
3,2,อังกฤษ,UK,Failed:country_code():Invalid code,country
4,2,อังกฤษ,UK,Failed:country_name():'อังกฤษ',country
5,2,อังกฤษ,UK,Passed:country_name_thai(),country
6,3,invalid name,invalid name,Failed:country_code():Invalid code,country
7,3,invalid name,invalid name,Failed:country_name():'invalid name',country
8,3,invalid name,invalid name,Failed:country_name_thai():'invalid name',country
